In [19]:
import os
import sys
import json
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_from_disk
import argparse

from collections import defaultdict, OrderedDict, Counter

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from transformers import (
    AutoTokenizer,
    TrainingArguments,
    set_seed
)

sys.path.append('../')
from preprocessor import Preprocessor
from elasticsearch import Elasticsearch, helpers
from retriever.retriever_sparse_BM25 import SparseRetrieval

In [4]:
datasets = load_from_disk('/opt/ml/data/train_dataset')
datasets.cleanup_cache_files()

test_datasets = load_from_disk('/opt/ml/data/test_dataset')
test_datasets.cleanup_cache_files()

{'validation': 0}

In [7]:
total_question = datasets['validation']['question'] + test_datasets['validation']['question']

In [26]:
last_token = []
for q in total_question :
    token = q.split(' ')
    last_token.append(token[-1])
    if token[-1] == '?':
        print(q)

학생들이 일본인 선생들을 몰아내기 위해 동맹휴학 시위를 일으킨 해는 ?


In [21]:
last_counter = Counter(last_token)
last_counter.most_common()

[('사람은?', 51),
 ('것은?', 50),
 ('인물은?', 40),
 ('곳은?', 31),
 ('누구인가?', 28),
 ('무엇인가?', 28),
 ('해는?', 24),
 ('이름은?', 15),
 ('어디인가?', 13),
 ('시기는?', 13),
 ('연도는?', 12),
 ('있는가?', 12),
 ('때문인가?', 11),
 ('장소는?', 10),
 ('무엇인가요?', 10),
 ('나라는?', 9),
 ('언제인가?', 8),
 ('국가는?', 7),
 ('이유는?', 7),
 ('지역은?', 6),
 ('방법은?', 6),
 ('기관은?', 6),
 ('대상은?', 5),
 ('부대는?', 5),
 ('하는가?', 5),
 ('누구인가요?', 5),
 ('년도는?', 4),
 ('물건은?', 4),
 ('명칭은?', 4),
 ('것인가?', 4),
 ('집단은?', 4),
 ('회사는?', 3),
 ('날짜는?', 3),
 ('했는가?', 3),
 ('있나?', 3),
 ('결과는?', 3),
 ('선수는?', 3),
 ('방식은?', 3),
 ('왕은?', 3),
 ('직업은?', 3),
 ('기간은?', 3),
 ('직책은?', 3),
 ('책은?', 3),
 ('있나요?', 3),
 ('사용하는가?', 2),
 ('저서는?', 2),
 ('도시는?', 2),
 ('역은?', 2),
 ('기술은?', 2),
 ('소속팀은?', 2),
 ('종교는?', 2),
 ('관직은?', 2),
 ('되었는가?', 2),
 ('되었나?', 2),
 ('주었는가?', 2),
 ('출신인가요?', 2),
 ('시대는?', 2),
 ('위치는?', 2),
 ('형벌은?', 2),
 ('어디였는가?', 2),
 ('하나요?', 2),
 ('자들은?', 2),
 ('어디인가요?', 2),
 ('물질은?', 2),
 ('명인가?', 2),
 ('문서는?', 2),
 ('때는?', 2),
 ('언론은?', 2),
 ('누구였는가?', 2),
 ('되는

In [22]:
total_question

['처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 '스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?',
 '촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?',
 '로타이르가 백조를 구하기 위해 사용한 것은?',
 '의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?',
 '1945년 쇼와천황의 항복 선언이 발표된 라디오 방송은?',
 '징금수는 서양 자수의 어떤 기법과 같은 기술을 사용하는가?',
 '다른 과 의사들은 감염내과 전문의들로부터 어떤 것에 대해 조언을 받는가?',
 '루이 14세의 왕비 마리아 테래사는 어느 나라 공주인가?',
 '헤자즈 왕국이 실존했던 것은 언제까지인가?',
 '버드 교장이 5월의 여왕의 대안으로 제시한 것은?',
 "인형사'를 만들어낸 것으로 추측되는 사업의 이름은?",
 '멘데스가 요원들을 구하기 위해 간 도시는 어디인가?',
 '교과부의 행동에 화가나 여러명이 사직한 기구의 이름은?',
 '반대동맹이 공산당과 갈라서겠다고 얘기한 날은 언제인가?',
 '피어슨이 다시 의회를 해산했던 년도는?',
 '몽케가 죽은 뒤 쿠릴타이에서 대칸의 지위를 얻은 사람의 이름은?',
 '이흥구의 사법시험 이야기를 기사로 작성한 곳은?',
 '남북조 시대에서 이이 씨가 전쟁이 발생했을 때, 생활했던 장소는?',
 '박지훈은 1라운드에서 몇 순위를 차지했는가?',
 '데메카론에는 무엇을 풍자하는 이야기가 들어있나요?',
 '병에 걸려 죽을 확률이 약 25~50%에 달하는 유형의 질병은?',
 '설리반이 불만을 표시한 대상은 누구인가?',
 '베소스는 어디서 추방당했는가?',
 '진전사의 명칭이 드러나는 데 영향을 준 물건은?',
 '자신의 이상적인 국가관이 스파르타와 닮아 있다고 생각하는 플라톤의 저서는?',
 '박제된 북극곰이 사망한 날짜는?',
 '문법 측면에서 더 보수적인 포르투갈어 표준은?',
 '로스 수장이 살해한 사람은 어느 당 회원인가?',
 '조경숙왕의 아들인 요자의 친어머니는 누구

In [37]:
args = argparse.Namespace(
    dataset_name="/opt/ml/data",
    model_name_or_path="klue/bert-base",
    data_path="/opt/ml/data",
    context_path="wikipedia_documents.json",
    save_dir="./encoders",
    num_neg=0,  
    device="cuda:0",
    reconfig='False'
)

dense_args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

set_seed(dense_args.seed)

datasets = load_from_disk('/opt/ml/data/train_dataset') 
val_df = pd.DataFrame(datasets["validation"])

model_checkpoint = args.model_name_or_path  # klue/bert-base
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/conda/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [37]:
import torch
a = torch.rand(2,3)
a.size(), a, torch.max(a, 1)

(torch.Size([2, 3]),
 tensor([[0.5183, 0.1832, 0.9921],
         [0.7888, 0.9984, 0.8023]]),
 torch.return_types.max(
 values=tensor([0.9921, 0.9984]),
 indices=tensor([2, 1])))

In [38]:
with open(os.path.join(args.dataset_name, args.context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)
wiki_contexts = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로
print(f"Lengths of unique contexts : {len(wiki_contexts)}")

Lengths of unique contexts : 56737


In [39]:
tokenizer.add_tokens(['화엄종','《악마는 프라다를 입는다》', '기렌 총수', '마리아', "\'형제의 나라\'"])

4

In [40]:
BM25 = SparseRetrieval(
    tokenize_fn=tokenizer.tokenize,
    data_path=args.dataset_name,
    context_path=args.context_path,
)
BM25.get_sparse_BM25()

Lengths of unique contexts : 56737
Build passage BM25_class_instant


Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors



BM25_class_instant pickle saved.


In [24]:
BM25_rank_ret = defaultdict(list)
BM25_rank_scores = defaultdict(list)
for q in tqdm(datasets["validation"]) :
    scores, indices = BM25.get_relevant_doc_BM25(q['question'], k=len(wiki_contexts))
    for r, idx in enumerate(list(indices)) :
        if q['context'] == wiki_contexts[idx]:
            BM25_rank_ret[r].append(q['question']+' '+q['answers']['text'][0]+' '+q['context'])
            BM25_rank_scores[r].append(scores[r])
            break

In [55]:
BM25_rank_ret_sorted = OrderedDict(sorted(BM25_rank_ret.items(), key = lambda t : t[0]))

for rank in BM25_rank_ret_sorted.keys() :
    print(rank, len(BM25_rank_ret_sorted[rank]))

0 133
1 42
2 13
3 5
4 2
5 5
6 7
7 4
8 5
9 1
11 1
12 1
15 2
16 3
19 1
23 2
25 1
26 1
45 1
54 1
66 1
164 1
174 1
178 1
493 1
505 1
1144 1
2164 1
3367 1


In [ ]:
tokenizer.tokenize('루이 14세의 왕비 마리아 테래사는 어느 나라 공주인가?')

In [ ]:
BM25_rank_ret[r].append(q['question']+' '+q['answers']['text'][0]+' '+q['context'])

In [61]:
one_ret = defaultdict(list)
scores, indices = BM25.get_relevant_doc_BM25('카르가 반대입장으로 돌아서자 반란세력이 진행한 것은?', k=len(wiki_contexts))
for r, idx in enumerate(list(indices)) :
    one_ret[r].append(str(scores[r]) + wiki_contexts[idx])
        # BM25_rank_scores[r].append(scores[r])

In [59]:
tokenizer.tokenize("카르가 반대입장으로 돌아서자 반란세력이 진행한 것은?")

["'형제의 나라'"]

In [ ]:
one_ret

In [ ]:
for rank in BM25_rank_scores.keys() :
    print(rank, BM25_rank_scores[rank])

In [296]:
for rank in BM25_rank_scores.keys() :
    if rank > 10 :
        for res in BM25_rank_ret_sorted[rank] :
            print(rank, res)

11 57.41750096602518 카르가 반대입장으로 돌아서자 반란세력이 진행한 것은? 데코행진 1923년 11월 8일 밤 바이에른 주지사 구스타브 리터 폰 카르는 뷰르가브로이케라의 맥주홀에서 바이에른의 실력자들을 모아 대집회를 열었고 오후 8시 15분에 카르의 연설이 시작되었다.\n\n그리고 에른스트 룀, 빌헬름 브뤼크너가 이끄는 SA의 단원 600명이 기관총으로 비어 홀을 포위하고 그의 동료 헤르만 괴링, 알프레트 로젠베르크, 루돌프 헤스, 에른스트 한스슈탱글, 울리히 그라프, 요한 아이그너, 아돌프 렌크, 막스 아만, 루드비히 막시밀리안 어윈 폰 스퀘부너 리히터, 빌헬름 아담, 율리우스 슈트라이허 등 나치당의 실력자들과 둘러싸인 히틀러가 등장했다.\n\n그리고 군중을 헤치고 8시 30분 경 프록코트를 입은 히틀러가 회장에 등단하여 천장을 향해 권총을 발사하면서 선언했다.\n\n국민혁명이 시작되었다. 회관은 6명의 사람들도 가득 차 있다. 베를린과 바이에른 정부는 지위를 잃었고 새 정부가 형성된다. 독일국방군과 경찰이 이들을 점유하고 있다.\n\n그리고 히틀러는 루돌프 헤스, 아돌프 렌크, 울리히 그라프와 함께 총으로 위협해 어안이 벙벙해 있는 카르, 세이서, 로소를 옆방으로 데리고 가 쿠데타 계획을 설명하고 반란 지원을 요구했다. 결국 루덴도르프 장군의 중재로 카르와 히틀러는 협력을 약속했으며 계획은 성공할 것처럼 보였다.\n\n한편 괴링은 바이에른 실력자들에게 떠나지 말라고 연설을 하나 일부는 주방을 통해 탈출했으며 하인츠 페르네, 요한 아이그너, 스퀘부너 리히터 등이 루덴도르프를 데리러 파견되고 룀은 도시의 주요 건물들을 점령하도록 명령했으며 공범자 게르하르트 로스바흐가 보병 학교의 학생들을 동원했다.\n\n그렇지만 무엇보다 우선 국방군이 히틀러를 지지하지 않았다. 베를린으로 진격하게 됨으로써 국방군들끼리 싸우게 되는 사태를 기피했기 때문이다. 관료들 역시 일치해서 카르에 대한 충성을 맹세했고 이렇게 해서 카르는 국방군의 보호하에 11월 9일 오전

In [56]:
for rank in BM25_rank_ret.keys() :
    for res in BM25_rank_ret_sorted[rank] :
        if '설리반이 불만을 표시한 대상은 누구인가?' in res :
            print(rank, res)
            break

2 설리반이 불만을 표시한 대상은 누구인가? 데스탱 장군 1778년 초, 설리번은 로드아일랜드로 전출되어 그곳의 부대와 민병대를 지휘하게 되었다. 그것은 프랑스의 참전이후 난공불락으로 여겨지는 뉴포트를 장악한 영국군을 프랑스 해군과 연계하여 공격하거나 포위하려는 의도였다. 그러나 데스탱 장군이 이끄는 함대가 폭우로 손실되어 그 계획은 취소되었다. 손실된 배와 하우 제독이 이끄는 영국 함대의 도착으로 데스탱은 보스톤으로 철수하였다. 그때 뉴포트의 영국군 요새에 있던 영국군이 출동하였고, 설리번도 8월에 일어난 영양가 없었던 로드아일랜드 전투에서 교전을 벌인 후 퇴각했다.\n\n난공불락으로 보였던 요새 공략에 실패하고, 작전이 붕괴된 방식으로 인해 프랑스와 미합중국 간의 관계가 불편해지게 되었다. 설리반은 데스탱 장군에게 편지를 써서 그가 목격했던 것을 ‘프랑스의 명예를 손상시키는’이라는 표현을 써가며 배반적이고, 비겁한 짓이라고 항의했다. 이 작전 실패로 두 연합국 간의 국제 분쟁에 불이 붙었으며, 1년 뒤 사바나 포위전에서 영국 요새에 대한 또 다른 실패를 불러왔다. 설리번은 이 실패로 그의 이력에 큰 타격을 입지는 않았으며, 그는 조심스런 캐나다 침공을 위한 지휘관으로 오르내리고 있었다.


In [57]:
BM25_rank_ret[1]

['처음으로 부실 경영인에 대한 보상 선고를 받은 회사는? 한보철강 순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.',
 '박지훈은 1라운드에서 몇 순위를 차지했는가? 전체 4순위 대전유천초등학교에서 야구를 시작할 당시에는 내야수였다. 한밭중학교 3학년 때 연습 경기에서 오버핸드 투수로 등판해 대량 실점을 했고, 감독의 지시에 의해 나머지 두 경기에서 사이드암으로 던져 모두 승리를 따낸 직후부터 사이드암 투수로 전향했다.\\n대전고등학교 3학년 때 비로소 팀의 에이스로 부각됐지만 구속이 느리다는 이유로 프로 구단에서 큰 관심을 보이지 않아 2008년 신인 드래프트에서 지명되지 못해 단국대학교에 진학했다. 진학 후 신장이 커져 구속을 140km대로 끌어올리는 데 성공하며 1학년 때부터 대학 최고 사이드암

{'validation': 0}

In [80]:
for a in zip(test_datasets['validation']['id'], test_datasets['validation']['question']) :
    print(a)

('mrc-1-000653', "유령'은 어느 행성에서 지구로 왔는가?")
('mrc-1-001113', '용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?')
('mrc-0-002191', '돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?')
('mrc-0-003951', '마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?')
('mrc-1-001272', '디엔비엔푸 전투에서 보응우옌잡이 상대한 국가는?')
('mrc-1-000993', '단공류가 일반 포유류와 다르다는 것을 알 수 있는 신체 부위는?')
('mrc-0-005021', '상류 부르주아의 정치 독점을 불러일으킨 체계는?')
('mrc-1-000163', '아시노 호가 만들어진 원인은 무엇인가?')
('mrc-0-001283', '흉년이 발생하고 곳곳에 난이 일어났던 시기는?')
('mrc-0-004543', '급전궤도는 보통 무엇으로 만들어져 있는가?')
('mrc-0-000439', '클레어에게 오세아닉 815편을 타라고 말한 사람의 이름은?')
('mrc-0-002895', '제6군이 18일 동안이나 대기해야 했던 장소는?')
('mrc-0-000535', '누가 시장으로 선출되면서 트랜짓 시티 계획안이 불발되었나?')
('mrc-1-001724', '1차 대각성 운동이 일어나는데 큰 영향을 끼친 설교는 어디서 진행되었는가?')
('mrc-0-000901', '괴수로부터 메구밍을 구해준 사람은 누구인가?')
('mrc-0-001606', '슈파이어 대성당 2차 축조시에 가장 우선순위로 고려했던 천장 건축 방식은?')
('mrc-0-000266', '통대를 뽑을 수 있었던 주체는 누구인가?')
('mrc-0-001326', '이유립씨가 1970년대 중반에 본인의 글을 기고하기 시작한 곳은?')
('mrc-0-000032', '미군이 충원되지 않으면 서울이 전쟁 시작 후 2주만에 함락될 것이라 밝힌 문서는?')
('mrc-0-005215', '임

In [89]:
test_one_ret = defaultdict(list)
scores, indices = BM25.get_relevant_doc_BM25('티칼왕조의 창건자는 몇 번 무덤에 묻혔다고 알려져있나?', k=len(wiki_contexts))
for r, idx in enumerate(list(indices)) :
    test_one_ret[r].append(str(scores[r]) + wiki_contexts[idx])

In [90]:
tokenizer.tokenize('티칼왕조의 창건자는 몇 번 무덤에 묻혔다고 알려져있나?')

['티',
 '##칼',
 '##왕',
 '##조',
 '##의',
 '창건',
 '##자',
 '##는',
 '몇',
 '번',
 '무덤',
 '##에',
 '묻',
 '##혔',
 '##다고',
 '알려져',
 '##있',
 '##나',
 '?']

In [91]:
test_one_ret

defaultdict(list,
            {0: ["87.31557218943821번 무덤\\n\\n1번 무덤은 잃어버린 세계 건물군에 있는 무덤이다. 이 무덤에서 아름다운 도자기 유물들이 출토되었고, 새의 머리와 목이 도자기에 그려진 몸체에서 입체적으로 뻗어나오고 있는 모습을 한 도자기가 특히 눈여겨볼 만한 유물이다.\\n 10번 무덤\\n\\n10번 무덤은 티칼의 왕 약스 누운 아이인의 무덤이다. 북부 아크로폴리스에 위치하며 속에 다양한 부장품들이 함께 발굴되었다. 이들 중에는 도자기와 음식들도 있었고, 특히 죽은 왕을 위하여 9명의 소년소녀들이 함께 산채로 바쳐졌다. 이때 강아지도 함께 묻혔다. 무덤 내부에서 발견된 도자기에는 티칼 양식과 테오티우아칸의 양식이 혼재되어 있으며, 이 곳에서 발견된 주요 부장품들 가운데 하나는 죽음의 신의 모습을 하고 있는 향로이다. 이 향로에는 사람의 뼈로 만들어진 받침대에 앉은채 잘린 사람 머리를 손에 들고 있는 죽음의 신이 생생하게 묘사되어 있다. 이 무덤은 자갈을 채워넣은 채로 봉인되었으며, 이후에 그 위에 피라미드가 세워져 도굴꾼들의 출입을 방지하는 방식으로 신중을 기했다.\\n\\n48번 무덤\\n\\n48번 무덤은 일반적으로 시야즈 찬 카이이 1세의 무덤으로 여겨진다. 북부 아크로폴리스 33번 신전 아래에 자리하며, 이 무덤은 기반암 속으로 파여있으며 왕의 유해와 함께 바쳐진 2명의 소년들의 유해가 있다. 무덤의 벽에는 백색 스투코가 칠해져 있으며 그 위에 화려한 상형문자들이 그림그려져 있다. 이 상형문자에는 457년 3월 20일까지를 나타내는 달력이 있는데, 아마도 이 왕이 죽은 날로 생각된다. 왕의 두개골과 뼈 대부분은 사라졌으나, 왕의 대퇴골과 손뼈 1개는 회수할 수 있었다. 다만 왕과 함께 묻힌 2명의 소년들의 유해는 완전한 모습으로 발견된 것으로 보아, 아마도 도굴꾼들의 행적이 이 곳에 닿았던 것으로 여겨진다.\\n\\n85번 무덤\\n\\n선고전기 후기에 만들어졌으며 기초적인 모습의 자갈을